In [5]:
import tensorflow as tf
import numpy as np
import keras

In [6]:
from keras.datasets import cifar10

In [7]:
num_classes = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
class BatchGenerator(object):
    def __init__(self, _X,_Y):
        self.X = _X
        self.Y = _Y
        self.cursor = 0
        self.length = len(_X)
        
    def next_batch(self, batch_size):
        flag = False
        
        if self.cursor + batch_size >= self.length:
            self.cursor = self.length - batch_size
            flag = True
            
        batch_x = self.X[self.cursor:self.cursor+batch_size]
        batch_y = self.Y[self.cursor:self.cursor+batch_size]
        
        if flag == True:
            self.cursor = 0
        else:
            self.cursor = self.cursor + batch_size
        
        return (batch_x,batch_y)

In [9]:
x = tf.placeholder(tf.float32, shape=[None, 32*32])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, shape=[-1,32,32,3])

#convolution layer1
w1=tf.Variable(tf.random_normal([5,5,3,64], stddev = 0.1))
b1=tf.Variable(tf.random_normal([64], stddev = 0.1))

conv_layer1 = tf.nn.relu(tf.nn.conv2d(x_image, w1, strides=[1,1,1,1], padding = 'SAME')+ b1)
#32,32,64,3

#pooling layer1 (16,16,64)
pool_layer1 = tf.nn.max_pool(conv_layer1, ksize=[1,3,3,1], strides = [1,2,2,1],padding = 'SAME')

#convolution layer2
w2=tf.Variable(tf.random_normal([5,5,64,64], stddev = 0.1))
b2=tf.Variable(tf.random_normal([64], stddev = 0.1))

conv_layer2 = tf.nn.relu(tf.nn.conv2d(pool_layer1, w2, strides=[1,1,1,1], padding = 'SAME')+b2)

#pooling layer2
pool_layer2 = tf.nn.max_pool(conv_layer2, ksize=[1,3,3,1], strides = [1,2,2,1],padding = 'SAME')
#8,8,64

#Fully connected layer 1
w3=tf.Variable(tf.random_normal([8*8*64, 1024], stddev = 0.1))
b3=tf.Variable(tf.random_normal([1024], stddev = 0.1))

flat = tf.reshape(pool_layer2, [-1, 8*8*64])

fc_layer1 = tf.nn.relu(tf.matmul(flat,w3)+b3)

#Fully connected layer 2
w4=tf.Variable(tf.random_normal([1024, 10], stddev = 0.1))
b4=tf.Variable(tf.random_normal([10], stddev = 0.1))

fc_layer2 = tf.matmul(fc_layer1,w4)+b4

y_conv = tf.nn.softmax(fc_layer2)

In [10]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits=fc_layer2)

In [11]:
train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

In [12]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch=next_batch(50)
        sess.run(train, feed_dict={x:batch[0], y_:batch[1]})
        
        if i%100 ==0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_:batch[1]})
            print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(accuracy, feed_dict={x:x_test, y_:y_test})

InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 11720130560